
## Search through Profile properties summaries using Jinja2 and Python Modele

*USE Python 3.7 to maintain order of Json files*

- Fetch SD file from IG
- Spec_internals from IG Package.tgz
- search for element properties



*Note need a successful build to generate since based on ig output local file
alternatively use package.files to generate*

### import python modules including R4 fhirclient models

In [69]:
from fhirclient.r4models import structuredefinition as SD
import Summary_maker.title_map as tm
import os, json, copy
from pathlib import Path

### Get file and return as dict

### Write to file

### Loop through profiles in source dir

In [70]:
duplicate_elements = set()

#us_core = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev/package'
us_core = '/Users/ehaas/Documents/FHIR/US-Core-R4/input/resources'
fhir_r4 = '/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package'
#f_name = 'us-core-patient'
#n_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev' # package file keeps disappearing
#in_path = '//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/output'  # use local build dir.
us_core_files = [x for x in os.listdir(us_core) if x.startswith("StructureDefinition")]
us_core_files

remove_list = ['alias', 'definition', 'isModifier', 'max', 'min', 'type']

In [71]:
def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}', encoding="utf8") as f:
        r = f.read()
        return(json.loads(r))

In [72]:
def save_to_file(sd_dict,f_name): # save in resources folder

    print('writing to file .......')
    write_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/input/resources'
    #write_path = 'test_resources'
    path = Path.cwd() / write_path  / f_name
    path.write_text(json.dumps(sd_dict, indent=2))

loop through all sd source files and compare to the fhir package snapshot

print out the element the element you want to scrub out

filter on the element properties you want

In [73]:
for i in us_core_files:
    print(i)
    constraints = {}
    sd_dict = open_file(us_core,i)
    if sd_dict['type'] != 'Extension':
        print('*'*79)
        print(f'******** {sd_dict["title"]} ************')
        print('*'*79)
        print(f"base = {sd_dict['baseDefinition']} type = {sd_dict['type']}")
        #get base SD
        if sd_dict['baseDefinition'] == 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-vital-signs':
            base_dict = open_file(us_core,'StructureDefinition-us-core-vital-signs.json')
            element_backbone = "differential"
        elif sd_dict['baseDefinition'] == 'http://hl7.org/fhir/StructureDefinition/vitalsigns':
            base_dict = open_file(fhir_r4,'StructureDefinition-vitalsigns.json')
            element_backbone = "snapshot"
            break
        else:
            base_dict = open_file(fhir_r4,f'StructureDefinition-{sd_dict["type"]}.json')
            element_backbone = "snapshot"
        assert base_dict['url'] == sd_dict['baseDefinition']

        #print(fhir_r4_sd.name,fhir_r4_sd.url) 
#         print(sd_dict['differential'].as_json())
        diff_copy = copy.deepcopy(sd_dict['differential']['element'])
        for el_index,e in enumerate(diff_copy):  #loop through all element properties in  and delete items from original
            print(f'******** Element = {e["path"]} ************')
#             print("Before:", json.dumps(e, indent=2))
            for j in base_dict[element_backbone]['element']:  # loop through the base to find matching element
                    if e['id'] == j['id']:
                        base_e = j
            for k,v in e.items():
     
                #print(f'k={k}, base_e[k]= {base_e[k]}')
                try:
                    assert k not in ['id','path']
                    assert k in remove_list
                    assert v==base_e[k]
                    print(f'{k} = {v} IS THE SAME AS BASE = {base_e[k]}')
                    sd_dict['differential']['element'][el_index].pop(k)

                except KeyError as r:
                    pass
#                     print(f'k={k} NOT THE SAME')
#                     print(r)
                except AssertionError as r:
                    pass
#                     print(f'k={k} NOT THE SAME')
#                     print(r)
                # loop through the base to see if different
#             print("After", json.dumps( sd_dict['differential']['element'][el_index], indent=2))
        save_to_file(sd_dict,i)  


StructureDefinition-us-core-observation-lab.json
*******************************************************************************
******** US Core Laboratory Result Observation Profile ************
*******************************************************************************
base = http://hl7.org/fhir/StructureDefinition/Observation type = Observation
******** Element = Observation ************
******** Element = Observation.status ************
******** Element = Observation.category ************
******** Element = Observation.category ************
******** Element = Observation.code ************
******** Element = Observation.subject ************
******** Element = Observation.effective[x] ************
******** Element = Observation.value[x] ************
******** Element = Observation.dataAbsentReason ************
writing to file .......
StructureDefinition-us-core-careteam.json
*******************************************************************************
******** US Core CareTeam 